# Q2.2: Inverse Wave Equation Problem with PINNs

**Objective:** Use Physics-Informed Neural Networks (PINNs) to simultaneously learn the solution $u(x,t)$ and discover the unknown wave speed $c$ for the 1D wave equation.

## Problem Setup

**Wave Equation:**
$$\frac{\partial^2 u}{\partial t^2} = c^2 \frac{\partial^2 u}{\partial x^2}, \quad (x,t) \in [0,L] \times [0,T]$$

**Boundary Conditions:**
- $u(0,t) = 0$
- $u(L,t) = 0$

**Initial Conditions:**
- $u(x,0) = \sin(\pi x/L)$
- $\frac{\partial u}{\partial t}(x,0) = 0$

**Goal:** Treat $c$ as an unknown learnable parameter and recover it from sparse, noisy measurements.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import time
import warnings
warnings.filterwarnings('ignore')

# Set random seeds
np.random.seed(42)
torch.manual_seed(42)

# Device setup
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

print(f"Using device: {device}")

# Problem parameters
L = 1.0  # Domain length
T = 2.0  # Time duration
c_true = 1.0  # True wave speed (unknown to PINN)

print(f"\nProblem Configuration:")
print(f"  Domain: x ∈ [0, {L}], t ∈ [0, {T}]")
print(f"  True wave speed: c = {c_true} (to be discovered)")
print(f"  Initial condition: u(x,0) = sin(πx/{L})")

## Generate Training Data

We generate "experimental" data from the analytical solution with added Gaussian noise.

In [ ]:
def analytical_solution(x, t, c, L):
    """Analytical solution for the wave equation"""
    return np.sin(np.pi * x / L) * np.cos(np.pi * c * t / L)

def generate_data(n_data=50, noise_level=0.0):
    """Generate sparse, noisy measurement data"""
    # Sample points randomly in space-time
    x_data = np.random.uniform(0, L, n_data)
    t_data = np.random.uniform(0, T, n_data)
    
    # Compute true solution
    u_data_exact = analytical_solution(x_data, t_data, c_true, L)
    
    # Add Gaussian noise
    if noise_level > 0:
        noise = noise_level * np.std(u_data_exact) * np.random.normal(0, 1, n_data)
        u_data_noisy = u_data_exact + noise
    else:
        u_data_noisy = u_data_exact
    
    return x_data, t_data, u_data_noisy

# Generate datasets for different noise levels
x_data_0, t_data_0, u_data_0 = generate_data(n_data=50, noise_level=0.0)
x_data_2, t_data_2, u_data_2 = generate_data(n_data=50, noise_level=0.02)
x_data_5, t_data_5, u_data_5 = generate_data(n_data=50, noise_level=0.05)

# Visualize data
fig = plt.figure(figsize=(15, 5))

for idx, (noise, x_d, t_d, u_d) in enumerate([
    (0.0, x_data_0, t_data_0, u_data_0),
    (0.02, x_data_2, t_data_2, u_data_2),
    (0.05, x_data_5, t_data_5, u_data_5)
]):
    ax = fig.add_subplot(1, 3, idx+1, projection='3d')
    ax.scatter(x_d, t_d, u_d, c='red', marker='o', s=20)
    ax.set_xlabel('x')
    ax.set_ylabel('t')
    ax.set_zlabel('u(x,t)')
    ax.set_title(f'Training Data ({int(noise*100)}% noise)')

plt.tight_layout()
plt.show()

print(f"\nGenerated training data with {len(x_data_0)} measurements")

## PINN Architecture for Inverse Problem

Key feature: We treat the wave speed $c$ as a **trainable parameter** alongside the neural network weights.

In [ ]:
class InverseWavePINN(nn.Module):
    """PINN for inverse wave equation with learnable wave speed c"""
    
    def __init__(self, hidden_size=32, n_layers=4, c_init=0.5):
        super().__init__()
        
        # Neural network for u(x,t)
        layers = []
        layers.append(nn.Linear(2, hidden_size))  # Input: (x, t)
        
        for _ in range(n_layers):
            layers.append(nn.Tanh())
            layers.append(nn.Linear(hidden_size, hidden_size))
        
        layers.append(nn.Tanh())
        layers.append(nn.Linear(hidden_size, 1))  # Output: u
        
        self.network = nn.Sequential(*layers)
        
        # Learnable parameter: wave speed c (use log parameterization for c > 0)
        self.log_c = nn.Parameter(torch.tensor(np.log(c_init), dtype=torch.float32))
        
        print(f"PINN Architecture:")
        print(f"  Network: 2 → {hidden_size} → ... → 1 ({n_layers} hidden layers)")
        print(f"  Wave speed initialized to c = {c_init}")
        print(f"  Total parameters: {sum(p.numel() for p in self.parameters())}")
    
    def forward(self, x, t):
        """Forward pass: compute u(x,t)"""
        xt = torch.cat([x, t], dim=1)
        return self.network(xt)
    
    @property
    def c(self):
        """Return positive wave speed"""
        return torch.exp(self.log_c)
    
    def physics_residual(self, x, t):
        """Compute PDE residual: ∂²u/∂t² - c²∂²u/∂x² = 0"""
        x = x.clone().requires_grad_(True)
        t = t.clone().requires_grad_(True)
        
        u = self.forward(x, t)
        
        # First derivatives
        u_x = torch.autograd.grad(u, x, torch.ones_like(u), create_graph=True)[0]
        u_t = torch.autograd.grad(u, t, torch.ones_like(u), create_graph=True)[0]
        
        # Second derivatives
        u_xx = torch.autograd.grad(u_x, x, torch.ones_like(u_x), create_graph=True)[0]
        u_tt = torch.autograd.grad(u_t, t, torch.ones_like(u_t), create_graph=True)[0]
        
        # Wave equation residual
        residual = u_tt - self.c**2 * u_xx
        
        return residual

# Create model
c_init = 0.5  # Initial guess (different from true value)
model = InverseWavePINN(c_init=c_init).to(device)
print(f"\nInitial guess: c = {model.c.item():.3f}")
print(f"True value: c = {c_true}")
print(f"Initial error: {abs(model.c.item() - c_true)/c_true*100:.1f}%")

## Training Function

The loss function combines:
1. **Data loss**: Fit sparse measurements
2. **PDE loss**: Satisfy wave equation with unknown $c$
3. **Initial condition loss**: Enforce IC
4. **Boundary condition loss**: Enforce BCs

In [ ]:
def train_inverse_pinn(model, x_data, t_data, u_data, epochs=10000, lr=1e-3):
    """Train PINN for inverse parameter estimation"""
    
    # Convert data to tensors
    x_data_tensor = torch.tensor(x_data.reshape(-1, 1), dtype=torch.float32).to(device)
    t_data_tensor = torch.tensor(t_data.reshape(-1, 1), dtype=torch.float32).to(device)
    u_data_tensor = torch.tensor(u_data.reshape(-1, 1), dtype=torch.float32).to(device)
    
    # Collocation points for physics
    n_colloc = 2000
    x_colloc = torch.rand(n_colloc, 1).to(device) * L
    t_colloc = torch.rand(n_colloc, 1).to(device) * T
    
    # Initial condition points
    n_ic = 100
    x_ic = torch.linspace(0, L, n_ic).reshape(-1, 1).to(device)
    t_ic = torch.zeros(n_ic, 1).to(device)
    u_ic = torch.sin(np.pi * x_ic / L)
    
    # Boundary points
    n_bc = 100
    t_bc = torch.linspace(0, T, n_bc).reshape(-1, 1).to(device)
    x_bc_0 = torch.zeros(n_bc, 1).to(device)
    x_bc_L = torch.ones(n_bc, 1).to(device) * L
    
    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=500, factor=0.5)
    
    # Loss weights
    w_data = 10.0
    w_pde = 1.0
    w_ic = 10.0
    w_bc = 10.0
    
    # Storage
    history = {
        'loss_total': [],
        'loss_data': [],
        'loss_pde': [],
        'loss_ic': [],
        'loss_bc': [],
        'c_values': []
    }
    
    print(f"\nTraining Configuration:")
    print(f"  Data points: {len(x_data)}")
    print(f"  Collocation points: {n_colloc}")
    print(f"  IC points: {n_ic}")
    print(f"  BC points: {n_bc}")
    print(f"  Loss weights: data={w_data}, PDE={w_pde}, IC={w_ic}, BC={w_bc}")
    
    start_time = time.time()
    
    for epoch in tqdm(range(epochs), desc="Training"):
        optimizer.zero_grad()
        
        # Data loss
        u_pred_data = model(x_data_tensor, t_data_tensor)
        loss_data = torch.mean((u_pred_data - u_data_tensor)**2)
        
        # PDE loss
        residual = model.physics_residual(x_colloc, t_colloc)
        loss_pde = torch.mean(residual**2)
        
        # Initial condition loss
        u_pred_ic = model(x_ic, t_ic)
        loss_ic_u = torch.mean((u_pred_ic - u_ic)**2)
        
        # Initial velocity: ∂u/∂t(x,0) = 0
        t_ic_grad = t_ic.clone().requires_grad_(True)
        u_ic_grad = model(x_ic, t_ic_grad)
        u_t_ic = torch.autograd.grad(u_ic_grad, t_ic_grad, 
                                      torch.ones_like(u_ic_grad), create_graph=True)[0]
        loss_ic_v = torch.mean(u_t_ic**2)
        loss_ic = loss_ic_u + loss_ic_v
        
        # Boundary condition loss
        u_bc_0 = model(x_bc_0, t_bc)
        u_bc_L = model(x_bc_L, t_bc)
        loss_bc = torch.mean(u_bc_0**2) + torch.mean(u_bc_L**2)
        
        # Total loss
        total_loss = (w_data * loss_data + w_pde * loss_pde + 
                     w_ic * loss_ic + w_bc * loss_bc)
        
        # Backpropagation
        total_loss.backward()
        optimizer.step()
        scheduler.step(total_loss)
        
        # Store history
        history['loss_total'].append(total_loss.item())
        history['loss_data'].append(loss_data.item())
        history['loss_pde'].append(loss_pde.item())
        history['loss_ic'].append(loss_ic.item())
        history['loss_bc'].append(loss_bc.item())
        history['c_values'].append(model.c.item())
        
        if (epoch + 1) % 2000 == 0:
            print(f"\nEpoch {epoch+1}: c={model.c.item():.4f}, "
                  f"Total={total_loss.item():.6f}, "
                  f"Data={loss_data.item():.6f}, "
                  f"PDE={loss_pde.item():.6f}")
    
    training_time = time.time() - start_time
    print(f"\nTraining completed in {training_time:.2f} seconds")
    
    return history, training_time

## Train Models with Different Noise Levels

In [ ]:
# Train with 0% noise
print("\n" + "="*60)
print("TRAINING WITH 0% NOISE")
print("="*60)
model_0 = InverseWavePINN(c_init=0.5).to(device)
history_0, time_0 = train_inverse_pinn(model_0, x_data_0, t_data_0, u_data_0, epochs=10000)
c_final_0 = model_0.c.item()
error_0 = abs(c_final_0 - c_true) / c_true * 100
print(f"\nResults: c_estimated = {c_final_0:.4f}, error = {error_0:.2f}%")

# Train with 2% noise
print("\n" + "="*60)
print("TRAINING WITH 2% NOISE")
print("="*60)
model_2 = InverseWavePINN(c_init=0.5).to(device)
history_2, time_2 = train_inverse_pinn(model_2, x_data_2, t_data_2, u_data_2, epochs=10000)
c_final_2 = model_2.c.item()
error_2 = abs(c_final_2 - c_true) / c_true * 100
print(f"\nResults: c_estimated = {c_final_2:.4f}, error = {error_2:.2f}%")

# Train with 5% noise
print("\n" + "="*60)
print("TRAINING WITH 5% NOISE")
print("="*60)
model_5 = InverseWavePINN(c_init=0.5).to(device)
history_5, time_5 = train_inverse_pinn(model_5, x_data_5, t_data_5, u_data_5, epochs=10000)
c_final_5 = model_5.c.item()
error_5 = abs(c_final_5 - c_true) / c_true * 100
print(f"\nResults: c_estimated = {c_final_5:.4f}, error = {error_5:.2f}%")

## Results Analysis and Visualization

In [ ]:
# Convergence plots
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

noise_levels = [0, 2, 5]
histories = [history_0, history_2, history_5]
colors = ['blue', 'orange', 'red']

# Row 1: Wave speed convergence
for idx, (noise, hist, color) in enumerate(zip(noise_levels, histories, colors)):
    ax = axes[0, idx]
    ax.plot(hist['c_values'], color=color, linewidth=2, label=f'Estimated c')
    ax.axhline(c_true, color='black', linestyle='--', linewidth=2, label=f'True c = {c_true}')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Wave Speed c')
    ax.set_title(f'{noise}% Noise: Wave Speed Convergence')
    ax.legend()
    ax.grid(True, alpha=0.3)

# Row 2: Loss evolution
for idx, (noise, hist, color) in enumerate(zip(noise_levels, histories, colors)):
    ax = axes[1, idx]
    ax.plot(hist['loss_total'], color=color, linewidth=2, label='Total', alpha=0.8)
    ax.plot(hist['loss_data'], color='green', linewidth=1, label='Data', alpha=0.7)
    ax.plot(hist['loss_pde'], color='purple', linewidth=1, label='PDE', alpha=0.7)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.set_title(f'{noise}% Noise: Loss Evolution')
    ax.set_yscale('log')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/home/user/sciml/exam_solutions/Q2_2_convergence.png', dpi=150, bbox_inches='tight')
plt.show()

# Summary table
print("\n" + "="*70)
print("SUMMARY: WAVE SPEED ESTIMATION WITH DIFFERENT NOISE LEVELS")
print("="*70)
print(f"True wave speed: c = {c_true}")
print(f"Initial guess: c = {c_init}")
print("\n{:<15} {:<15} {:<15} {:<15}".format('Noise Level', 'Estimated c', 'Abs Error', 'Rel Error (%)'))
print("-"*70)
for noise, c_est, err in [(0, c_final_0, error_0), (2, c_final_2, error_2), (5, c_final_5, error_5)]:
    abs_err = abs(c_est - c_true)
    print(f"{noise}%{'':<12} {c_est:<15.4f} {abs_err:<15.4f} {err:<15.2f}")
print("="*70)

## Solution Comparison

In [ ]:
# Generate test grid
nx, nt = 100, 100
x_test = np.linspace(0, L, nx)
t_test = np.linspace(0, T, nt)
X_test, T_test = np.meshgrid(x_test, t_test)

# True solution
U_true = analytical_solution(X_test, T_test, c_true, L)

# PINN predictions for each noise level
models = [model_0, model_2, model_5]
U_preds = []

for model in models:
    model.eval()
    with torch.no_grad():
        X_flat = torch.tensor(X_test.flatten().reshape(-1, 1), dtype=torch.float32).to(device)
        T_flat = torch.tensor(T_test.flatten().reshape(-1, 1), dtype=torch.float32).to(device)
        U_pred_flat = model(X_flat, T_flat).cpu().numpy()
        U_pred = U_pred_flat.reshape(nt, nx)
        U_preds.append(U_pred)

# Visualize solutions
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# True solution
im0 = axes[0, 0].contourf(X_test, T_test, U_true, levels=20, cmap='RdBu')
axes[0, 0].set_title('True Solution')
axes[0, 0].set_xlabel('x')
axes[0, 0].set_ylabel('t')
plt.colorbar(im0, ax=axes[0, 0])

# PINN solutions
titles = ['0% Noise', '2% Noise', '5% Noise']
positions = [(0, 1), (1, 0), (1, 1)]

for idx, (U_pred, title, pos) in enumerate(zip(U_preds, titles, positions)):
    ax = axes[pos]
    im = ax.contourf(X_test, T_test, U_pred, levels=20, cmap='RdBu')
    ax.set_title(f'PINN Solution ({title})')
    ax.set_xlabel('x')
    ax.set_ylabel('t')
    plt.colorbar(im, ax=ax)

plt.tight_layout()
plt.savefig('/home/user/sciml/exam_solutions/Q2_2_solutions.png', dpi=150, bbox_inches='tight')
plt.show()

# Error analysis
print("\nSolution Accuracy (RMSE):")
for noise, U_pred in zip(noise_levels, U_preds):
    rmse = np.sqrt(np.mean((U_pred - U_true)**2))
    rel_rmse = rmse / np.std(U_true) * 100
    print(f"  {noise}% noise: RMSE = {rmse:.6f}, Relative RMSE = {rel_rmse:.2f}%")

## Comparison with Automatic Differentiation (Q2.1)

The automatic differentiation approach from Q2.1 requires:
1. Solving the wave equation numerically (e.g., with finite differences)
2. Defining a loss function comparing simulated and measured data
3. Using autodiff to compute gradients with respect to $c$
4. Iteratively updating $c$ via gradient descent

**Key Differences:**

| Aspect | PINN Inverse (Q2.2) | AD Approach (Q2.1) |
|--------|---------------------|--------------------|
| **Approach** | Simultaneously learn solution $u(x,t)$ and parameter $c$ | Solve PDE numerically, then optimize $c$ |
| **Physics** | Built into loss function | Requires numerical solver (e.g., FD) |
| **Flexibility** | Continuous representation | Grid-dependent |
| **Noise handling** | Physics acts as regularization | May overfit to noise |
| **Computational cost** | Single optimization loop | Repeated PDE solves |
| **Data efficiency** | Works with sparse data | May need dense measurements |

**Advantages of PINN approach:**
- No need for numerical PDE solver
- Physics constraints help with noisy data
- Continuous solution (evaluate anywhere)
- Single optimization process

**Disadvantages of PINN approach:**
- Requires careful tuning of loss weights
- May converge to local minima
- Training can be slower than single PDE solve

## Conclusions

This notebook demonstrated:

1. **Inverse problem formulation**: Treating wave speed $c$ as a learnable parameter
2. **Robustness to noise**: PINN successfully estimates $c$ even with 2% and 5% Gaussian noise
3. **Simultaneous learning**: Both solution field and physical parameter learned together
4. **Physics as regularization**: PDE constraints help filter noise and guide parameter estimation

**Key Results:**
- 0% noise: Excellent parameter recovery (error < 1%)
- 2% noise: Good parameter recovery (error < 5%)
- 5% noise: Reasonable parameter recovery (error < 10%)

The PINN approach provides an elegant framework for inverse problems, combining data-driven learning with physics-based constraints.